In [2]:
from pypbc import *
import hashlib
import time

In [3]:
stored_params = """type a
q 8780710799663312522437781984754049815806883199414208211028653399266475630880222957078625179422662221423155858769582317459277713367317481324925129998224791
h 12016012264891146079388821366740534204802954401251311822919615131047207289359704531102844802183906537786776
r 730750818665451621361119245571504901405976559617
exp2 159
exp1 107
sign1 1
sign0 1
"""

params = Parameters(param_string=stored_params)
pairing = Pairing(params)

In [4]:
def setup():
    g = Element.random(pairing, G1)
    
    return g

g = setup()
g

0367EFF9947101C58C4F40F0F8A003A470925E248E862115C166EBDC40A4A302DC82CD7DF08A33AA55C2B90A8EE614368624EB2CC6E0D69ACABB0C04293F473A3C

In [5]:
def keygen(g):
    x = Element.random(pairing, Zr)
    y = g**x
    
    return x, y

x, y = keygen(g)
y

025E5FDF8A1B2EDEF906BA75B95880C92400A1F5D86FF6C59E55B208A16461C430BD72F700209120F672DB07EAB3908B366AC83695B3F0939A66DEAC627AEF3B89

In [6]:
def get_hash(c1, c2, c3):
        h_in = str(c1) + str(c2) + str(c3)
        sha512 = hashlib.sha512()
        sha512.update(h_in.encode('utf8'))
        hv = sha512.hexdigest()
        ev = Element.from_hash(pairing, G1, hv)

In [10]:
def enc(m, g, y):
    r = Element.random(pairing, Zr)
    u = g**r
    v = m**r
    
    h = get_hash(u, v, y**r)
#     mr_s = str(m+r)
#     mr = Element(pairing, G1, value=mr_s)
    w = h
    
    return [u, v, w]
    
c1 = []
c2 = []
x1, y1 = keygen(g)
x2, y2 = keygen(g)

start = time.time()
for i in range(9):
    m = Element.random(pairing, G1)
    c1.append(enc(m, g, y1))
    c2.append(enc(m, g, y2))
enc_time = time.time() - start
print(enc_time)

0.40356993675231934


In [11]:
def test(c1, c2):
    if pairing.apply(c1[0], c2[1]) == pairing.apply(c2[0], c1[1]):
        return 1
    else:
        return 0

d = []
start = time.time()
for i in range(len(c1)):
    d.append(test(c1[i], c2[i]))
test_time = time.time() - start
print(test_time)
print(d)

0.03622603416442871
[1, 1, 1, 1, 1, 1, 1, 1, 1]


In [21]:
p = Element.random(pairing, G1)
q = Element.random(pairing, G1)
print(p)
print(q)

ps = int(str(p), 16)
qs = int(str(q), 16)
print(ps)
print(qs)
rv = ps ^ qs
rr = hex(rv)
print(rr)
r = Element(pairing, G1, value=str(rr))
r

0313B9185148EA20AEBB89FDD2738602BD72315C934436813BA8A1BFF8E5907A3B47D60E170C984652662F13B8A56ADCAE7ACCE899AAF2E8835373542EC58593FF
0294BDD5D62B0C188EAC073458426C8E0AF0491103915F6A05F6969D9BEC17840CFDDB389D9F888A23384A31CA90638AD4246847F7472EC1E17C1EF7C669E4C0FC
41256402547305887525688048417597474873816208019603129600556031317969067596695188002622651863306887992638009186539408095521638529312145353691256177488663551
34605842637223105101664704448574568148637782037197953491204567051260414958866553811482818389091685865328073845189448653951918690701910685768621160283816188
0x18704cd8763e63820178ec98a31ea8cb782784d90d569eb3e5e3722630987fe37ba0d368a9310cc715e6522723509567a5ea4af6eeddc29622f6da3e8ac615303


0330354636393639443942454331373834304346444442333839443946383838413233333834413331434139303633384144343234363834374637343732454331